# Práctica de Laboratorio de Procesamiento del Lenguaje Natural (NLP)

Tema: Clustering divisivo mediante K-means

## Objetivo de la práctica

El objetivo de esta práctica es procesar los datos de entrada dados en el "corpus", con el objetivo de ejecutar clustering divisivo (utilizando `K-means`) para comprobar los resultados.



In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

pd.options.display.max_colwidth = 200
%matplotlib inline

## Corpus de documentos

In [ ]:
corpus = ['El cielo es azul y luminoso.',
          '¡Me encanta este cielo azul y luminoso!',
          'El zorro marrón es rápido y salta sobre el perro que es dormilón.',
          'Un desayuno real tiene salchichas, jamón, bacon, huevos, tostadas y queso',
          '¡Me encanta el jamón, los huevos, las salchichas y el bacon!',
          '¡El zorro marrón es rápido, y el perro azul es un dormilón!',
          'El cielo es azul intenso y hoy está muy luminoso',
          '¡El perro es un dormilón, pero el zorro es mu rápido!'
]

corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Documento': corpus}) 

corpus_df

# Pre-procesamiento

Para poder clusterizar los textos con mayor facilidad, es necesario que se queden las palabras que contengan significado de cada texto. Por ello, preprocesa los textos para eliminar los caracteres especiales, unificarlo en minúsculas, eliminar espacios extra (utiliza `strip()` para esto) y eliminar las stopwords.

In [ ]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('spanish')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc



In [ ]:
norm_corpus=[]

for document in corpus:
    norm_corpus.append(normalize_document(document))

norm_corpus

# Generación de las matrices

Genera BOW, TFIDF & Bag Of N-grams; se utilizarán para comparar los resultados.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# get bag of words features in sparse format
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix

In [ ]:
# view dense representation 
# warning might give a memory error if data is too big
cv_matrix = cv_matrix.toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tt = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True)
tt_matrix = tt.fit_transform(cv_matrix)

tt_matrix = tt_matrix.toarray()
vocab = cv.get_feature_names()
pd.DataFrame(np.round(tt_matrix, 2), columns=vocab)

# Kmeans

Utiliza KMeans para clusterizar los textos en distintos grupos. El número de clusters será 4. ¿Observas alguna diferencia con el Clustering jerárquico?

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 3

km = KMeans(num_clusters)
km.fit(tt_matrix)

cluster_labels = km.labels_.tolist()
pd.concat([corpus_df, pd.DataFrame(cluster_labels)], axis=1)